# 🖥️ MNIST Experiment
#
In this notebook, we train:
1. The Bayes by Backprop model on MNIST for every combination of (π, σ₁, σ₂).  
2. A standard feedforward neural network (two hidden layers) optimized with SGD and no regularization.  
3. The same feedforward network, but with Dropout applied to both hidden layers, still optimized with SGD.  
#
During each training loop, we display progress bars for both epochs and batches, along with printed outputs of key metrics.

### 🔧 Environment Setup

This section initializes the environment for training a PyTorch model:

- Imports necessary libraries including PyTorch, NumPy, and torchvision for data handling.
- Uses `tqdm` for progress bars during training.
- Automatically selects the available device: GPU (`cuda`) if present, otherwise defaults to CPU.
- Prints the selected device to confirm which hardware will be used for computations.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import trange, tqdm
import math

# Set the device (GPU if available, otherwise CPU)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"🚀 Using device: {device}")

🚀 Usando device: cuda


### 📚 Dataset Preparation: TensorFlow and PyTorch Versions of MNIST

This section performs the following:

#### TensorFlow
- Loads the MNIST dataset using `tf.keras.datasets`.
- Normalizes pixel values to [0, 1].
- Flattens 28×28 images into 784-dimensional vectors.

#### PyTorch
- Downloads and transforms the MNIST dataset using `torchvision`.
- Applies `ToTensor()` and reshapes the images to 1D vectors.
- Sets up `DataLoader` objects for training and testing with appropriate batch sizes and performance optimizations.

#### Shared Hyperparameters
- Defines standard values for learning rate, batch size, input/output dimensions, and number of epochs.
- Logs dataset sizes and input dimensions for confirmation.

In [ ]:
# %% [code]
# 📦 TensorFlow + TF Probability
import tensorflow as tf

# Load MNIST dataset
(x_train_tf, y_train_tf), (x_test_tf, y_test_tf) = tf.keras.datasets.mnist.load_data()

# Normalize and reshape the data
x_train_tf = x_train_tf.astype('float32') / 255.0
x_test_tf  = x_test_tf.astype('float32') / 255.0
x_train_tf = x_train_tf.reshape(-1, 28 * 28)
x_test_tf  = x_test_tf.reshape(-1, 28 * 28)

print(f"✅ TF MNIST: {x_train_tf.shape[0]} train, {x_test_tf.shape[0]} test, input_dim = {x_train_tf.shape[1]}")

# %% [code]
# 🔥 PyTorch
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Transform: convert to Tensor and flatten
transform = transforms.Compose([
    transforms.ToTensor(),                              # Converts [0,255] to [0,1] and to FloatTensor C×H×W
    transforms.Lambda(lambda x: x.view(-1))             # Flattens 28×28 image to a vector of 784
])

# Download and prepare datasets
train_dataset_torch = datasets.MNIST(root='./data', train=True,  download=True, transform=transform)
test_dataset_torch  = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Create DataLoaders
BATCH_SIZE_TORCH = 512
train_loader_torch = DataLoader(train_dataset_torch, batch_size=BATCH_SIZE_TORCH,
                                shuffle=True,  num_workers=4, pin_memory=True)
test_loader_torch  = DataLoader(test_dataset_torch,  batch_size=1000,
                                shuffle=False, num_workers=2, pin_memory=True)

print(f"✅ Torch MNIST: {len(train_dataset_torch)} train, {len(test_dataset_torch)} test, input_dim = {28*28}")

# Hyperparameters
NUM_EPOCHS = 600
LR = 1e-3
BATCH_SIZE = 1024
INPUT_DIM = 28 * 28
HIDDEN_DIM = 800
OUTPUT_DIM = 10
INPUT_DIM = 28 * 28
N_TRAIN = len(train_dataset_torch)
N_TEST  = len(test_dataset_torch)
print(f"✅ Loaded MNIST: {N_TRAIN} train, {N_TEST} test, input_dim = {INPUT_DIM}")

2025-06-13 12:32:09.597452: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749817929.795861      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749817929.851427      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
✅ TF MNIST: 60000 train, 10000 test, input_dim = 784


100%|██████████| 9.91M/9.91M [00:00<00:00, 53.0MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.65MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 14.5MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 9.72MB/s]

✅ Torch MNIST: 60000 train, 10000 test, input_dim = 784
✅ Caricati MNIST: 60000 train, 10000 test, input_dim = 784


### 🧠 Model Definitions: Bayesian and Feedforward Networks

This section implements three neural network architectures for MNIST classification:

#### 1. `BayesianLinear` Layer
- Custom fully-connected layer with Bayesian treatment of weights.
- Learns distributions (`μ`, `ρ`) instead of point estimates.
- Samples weights during training and computes log posterior (`log_qw`).

#### 2. `BayesianNet`
- A 3-layer fully connected Bayesian neural network using `BayesianLinear`.
- Uses a mixture of Gaussians prior (π, σ₁, σ₂).
- During forward pass, samples weights and accumulates KL divergence for ELBO.

#### 3. `FFNet`
- A standard feedforward network with two hidden ReLU layers and no regularization.

#### 4. `FFNetDropout`
- Identical to `FFNet`, but applies Dropout (p=0.5) after each hidden layer to prevent overfitting.

#### Other Elements
- `log_mixture_prior`: computes log-probabilities under a mixture Gaussian prior.
- `train_loader` and `test_loader`: PyTorch `DataLoader` objects for efficient batching.
- `criterion`: Cross-entropy loss used for classification.

In [ ]:
# ------------------------------------------------------------
# (1) BayesianLinear Layer and BayesianNet Model
# ------------------------------------------------------------
class BayesianLinear(nn.Module):
    """
    Fully-connected Bayesian layer:
     - defines μ and ρ parameters for weights and biases
     - σ = log(1 + exp(ρ)) ensures positivity and stability
     - log_qw is computed during each forward pass
    """
    def __init__(self, in_features: int, out_features: int):
        super().__init__()
        self.in_features  = in_features
        self.out_features = out_features

        # Parameters μ and ρ
        self.mu_weight  = nn.Parameter(torch.zeros(out_features, in_features))
        self.rho_weight = nn.Parameter(torch.ones(out_features, in_features) * -3.0)
        self.mu_bias    = nn.Parameter(torch.zeros(out_features))
        self.rho_bias   = nn.Parameter(torch.ones(out_features) * -3.0)

        # log_qw will be stored after sampling
        self.log_qw = None

    def compute_log_qw(self, w: torch.Tensor, sigma_w: torch.Tensor) -> torch.Tensor:
        """
        Compute log q(w) = -0.5 * sum( log(2πσ²) + ((w - μ)² / σ²) )
        """
        return -0.5 * torch.sum(torch.log(2 * torch.pi * sigma_w**2) +
                                ((w - self.mu_weight) ** 2) / (sigma_w**2))

    def sample_weight_and_bias(self) -> (torch.Tensor, torch.Tensor, torch.Tensor):
        """
        Sample (weights, biases) and return log_qw for KL divergence calculation.
        """
        sigma_w = torch.log1p(torch.exp(self.rho_weight))
        sigma_b = torch.log1p(torch.exp(self.rho_bias))

        eps_w = torch.randn_like(self.mu_weight)
        eps_b = torch.randn_like(self.mu_bias)
        w = self.mu_weight + sigma_w * eps_w
        b = self.mu_bias   + sigma_b * eps_b

        log_qw = self.compute_log_qw(w, sigma_w)
        return w, b, log_qw

    def forward(self, x: torch.Tensor, sample: bool = True) -> torch.Tensor:
        """
        If sample=True and model is training, sample weights and biases;
        otherwise use mean values (μ). Computes and stores log_qw.
        """
        sigma_w = torch.log1p(torch.exp(self.rho_weight))
        sigma_b = torch.log1p(torch.exp(self.rho_bias))

        if self.training and sample:
            eps_w = torch.randn_like(self.mu_weight)
            eps_b = torch.randn_like(self.mu_bias)
            w = self.mu_weight + sigma_w * eps_w
            b = self.mu_bias   + sigma_b * eps_b
        else:
            w = self.mu_weight
            b = self.mu_bias

        self.log_qw = -0.5 * torch.sum(torch.log(2 * torch.pi * sigma_w**2) +
                                       ((w - self.mu_weight)**2) / (sigma_w**2))

        return F.linear(x, w, b)


def log_mixture_prior(w_flat: torch.Tensor, pi: float, sigma1: float, sigma2: float) -> torch.Tensor:
    """
    Compute log[ π N(0,σ1²) + (1−π) N(0,σ2²) ] for each element of w_flat,
    then return the total log-probability.
    """
    log_const1 = 0.5 * math.log(2 * math.pi * sigma1**2)
    log_const2 = 0.5 * math.log(2 * math.pi * sigma2**2)

    log_prob1 = -0.5 * (w_flat ** 2) / (sigma1 ** 2) - log_const1
    log_prob2 = -0.5 * (w_flat ** 2) / (sigma2 ** 2) - log_const2

    log_pi         = math.log(pi)
    log_1_minus_pi = math.log(1.0 - pi)

    t1 = log_prob1 + log_pi
    t2 = log_prob2 + log_1_minus_pi

    max_log = torch.max(t1, t2)
    log_mix = max_log + torch.log(torch.exp(t1 - max_log) + torch.exp(t2 - max_log))

    return log_mix.sum()


class BayesianNet(nn.Module):
    """
    Fully-connected Bayesian network with 3 layers (2 hidden + output).
    """
    def __init__(self, input_dim, hidden_dim, output_dim, pi, sigma1, sigma2):
        super().__init__()
        # Prior parameters
        self.pi     = pi
        self.sigma1 = sigma1
        self.sigma2 = sigma2

        # Bayesian layers
        self.bl1 = BayesianLinear(input_dim,  hidden_dim)
        self.bl2 = BayesianLinear(hidden_dim, hidden_dim)
        self.bl3 = BayesianLinear(hidden_dim, output_dim)

        # KL divergence accumulator
        self.kl = None

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        self.kl = 0.0

        ####### LAYER 1 #######
        w1, b1, log_qw1 = self.bl1.sample_weight_and_bias()
        w1_flat = torch.cat([w1.view(-1), b1.view(-1)])
        log_pw1 = log_mixture_prior(w1_flat, self.pi, self.sigma1, self.sigma2)
        self.kl += (log_qw1 - log_pw1)
        x1 = F.relu(F.linear(x, w1, b1))

        ####### LAYER 2 #######
        w2, b2, log_qw2 = self.bl2.sample_weight_and_bias()
        w2_flat = torch.cat([w2.view(-1), b2.view(-1)])
        log_pw2 = log_mixture_prior(w2_flat, self.pi, self.sigma1, self.sigma2)
        self.kl += (log_qw2 - log_pw2)
        x2 = F.relu(F.linear(x1, w2, b2))

        ####### OUTPUT LAYER #######
        w3, b3, log_qw3 = self.bl3.sample_weight_and_bias()
        w3_flat = torch.cat([w3.view(-1), b3.view(-1)])
        log_pw3 = log_mixture_prior(w3_flat, self.pi, self.sigma1, self.sigma2)
        self.kl += (log_qw3 - log_pw3)
        logits = F.linear(x2, w3, b3)

        return logits


# DataLoaders for training and testing
train_loader = DataLoader(
    train_dataset_torch,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,
    pin_memory=True
)
test_loader = DataLoader(
    test_dataset_torch,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

class FFNet(nn.Module):
    """
    Standard feedforward neural network with 2 hidden ReLU layers.
    """
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim,  hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class FFNetDropout(nn.Module):
    """
    Same architecture as FFNet, but with Dropout (p=0.5) after each hidden layer.
    """
    def __init__(self, input_dim, hidden_dim, output_dim, p: float = 0.5):
        super().__init__()
        self.fc1 = nn.Linear(input_dim,  hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(p)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

# Loss function
criterion = nn.CrossEntropyLoss()

### ⚙️ Hyperparameter Configuration

This section defines the training settings for two model types:

#### 🧪 Bayes by Backprop
- **Epochs**: 600
- **Learning rate**: 1e-3
- **Prior parameters**:
  - `π = 0.75`
  - `σ₁ = exp(−2)`
  - `σ₂ = exp(−6)`
- These are used in the mixture Gaussian prior for Bayesian layers.

#### 🧪 Baseline SGD (Standard & Dropout Networks)
- **Epochs**: 20
- **Learning rate**: 1e-3
- Same loss function (`CrossEntropyLoss`) is used for both Bayesian and baseline models.

The goal is to ensure a consistent and fair comparison between the Bayesian and non-Bayesian approaches.

In [ ]:
# Hyperparameters for Bayes by Backprop training
NUM_EPOCHS_BBB = 600
LR_BBB = 1e-3
PI_LIST = [0.75]
SIGMA1_LIST = [np.exp(-2)]
SIGMA2_LIST = [np.exp(-6)]

# Hyperparameters for baseline SGD models (aligned to BBB for fair comparison)
NUM_EPOCHS_SGD = 20
LR_SGD = 1e-3

# Loss function for classification
criterion = nn.CrossEntropyLoss()

# Display configured hyperparameters
print("ℹ️ Hyperparameters:")
print(f"  Bayes by Backprop: NUM_EPOCHS = {NUM_EPOCHS_BBB}, LR = {LR_BBB}")
print(f"    PI_LIST     = {PI_LIST}")
print(f"    SIGMA1_LIST = {SIGMA1_LIST}")
print(f"    SIGMA2_LIST = {SIGMA2_LIST}")
print(f"\n  Baseline SGD: NUM_EPOCHS = {NUM_EPOCHS_SGD}, LR = {LR_SGD}\n")

ℹ️ Iperparametri:
  Bayes by Backprop: NUM_EPOCHS = 600, LR = 0.001
    PI_LIST     = [0.75]
    SIGMA1_LIST = [0.1353352832366127]
    SIGMA2_LIST = [0.0024787521766663585]

  Baseline SGD: NUM_EPOCHS = 20, LR = 0.001



### 🤖 Training Feedforward Models in TensorFlow

This section implements and trains two neural network models using TensorFlow:

#### 🏗️ Model Definitions
- `build_sgd_model()`: A plain feedforward neural network with two hidden ReLU layers.
- `build_dropout_model()`: Same as above, but includes Dropout (p=0.5) after each hidden layer to act as a regularizer.

#### 🏋️ Training
- Both models are compiled with the Adam optimizer and trained using cross-entropy loss.
- A `train_model()` function wraps training logic and logs progress.
- Training is performed on the MNIST dataset using `validation_split=0.1`.

#### 📊 Evaluation
- After training, both models are evaluated on the test set.
- Final test accuracies are printed to compare:
  - **SGD without regularization**
  - **SGD with Dropout**

This setup provides a performance baseline for comparison against Bayesian models.

In [12]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np

# 1️⃣ Model construction
def build_sgd_model():
    return tf.keras.Sequential([
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10)  # Output layer (logits)
    ])

def build_dropout_model():
    return tf.keras.Sequential([
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10)  # Output layer (logits)
    ])

# 2️⃣ Training function
def train_model(model, name, x_train, y_train, epochs=10):
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )
    print(f"\n🔧 Training {name}")
    model.fit(x_train, y_train,
              batch_size=64,
              epochs=epochs,
              validation_split=0.1,
              verbose=2)
    print(f"✅ Finished training {name}")
    return model

# Train plain SGD model (no regularization)
sgd_model = train_model(build_sgd_model(), "SGD", x_train_tf, y_train_tf, epochs=NUM_EPOCHS_SGD)

# Train SGD + Dropout model
dropout_model = train_model(build_dropout_model(), "Dropout", x_train_tf, y_train_tf, epochs=NUM_EPOCHS_SGD)

# Evaluate both models on the test set
loss_sgd, acc_sgd = sgd_model.evaluate(x_test_tf, y_test_tf, verbose=0)
loss_do,  acc_do  = dropout_model.evaluate(x_test_tf, y_test_tf, verbose=0)

# Print results
print(f"📈 SGD without regularization Accuracy: {acc_sgd:.4f}")
print(f"📈 SGD + Dropout (p=0.5) Accuracy: {acc_do:.4f}")


🔧 Training SGD
Epoch 1/20
844/844 - 5s - 6ms/step - accuracy: 0.8852 - loss: 0.3968 - val_accuracy: 0.9475 - val_loss: 0.1928
Epoch 2/20
844/844 - 2s - 2ms/step - accuracy: 0.9418 - loss: 0.1987 - val_accuracy: 0.9578 - val_loss: 0.1517
Epoch 3/20
844/844 - 2s - 2ms/step - accuracy: 0.9553 - loss: 0.1520 - val_accuracy: 0.9637 - val_loss: 0.1238
Epoch 4/20
844/844 - 2s - 2ms/step - accuracy: 0.9635 - loss: 0.1239 - val_accuracy: 0.9677 - val_loss: 0.1136
Epoch 5/20
844/844 - 2s - 2ms/step - accuracy: 0.9679 - loss: 0.1066 - val_accuracy: 0.9660 - val_loss: 0.1145
Epoch 6/20
844/844 - 2s - 2ms/step - accuracy: 0.9720 - loss: 0.0918 - val_accuracy: 0.9678 - val_loss: 0.1123
Epoch 7/20
844/844 - 2s - 2ms/step - accuracy: 0.9750 - loss: 0.0825 - val_accuracy: 0.9715 - val_loss: 0.0985
Epoch 8/20
844/844 - 2s - 2ms/step - accuracy: 0.9768 - loss: 0.0726 - val_accuracy: 0.9735 - val_loss: 0.0952
Epoch 9/20
844/844 - 2s - 2ms/step - accuracy: 0.9794 - loss: 0.0654 - val_accuracy: 0.9717 - va

### 🔁 Training Bayesian Neural Networks with All Prior Combinations

This block performs a full sweep over different combinations of prior parameters for the **Bayes by Backprop** model:

#### 🔧 What happens:
- For each combination of:
  - π (mixing coefficient)
  - σ₁, σ₂ (standard deviations for the mixture prior)
- A new `BayesianNet` is initialized and trained on MNIST.
- The model uses:
  - **Adam** optimizer
  - **600 epochs** of training
  - **KL-divergence regularization** added to the negative log-likelihood

#### 🧠 Evaluation:
- After each epoch:
  - The model is evaluated on the test set using the **mean weights (μ)** (i.e., deterministic mode).
  - If the current test accuracy exceeds the best so far, it is stored.

#### 📦 Output:
- Metrics such as training loss, accuracy, and test accuracy are printed for every epoch.
- The best test accuracy for each parameter combination is stored in the `results_bbb` list.

This process helps identify which prior configuration leads to the most accurate Bayesian model.

In [ ]:
results_bbb = []  # Store (pi, sigma1, sigma2, best_test_accuracy)

# Iterate over combinations of prior parameters
for pi in PI_LIST:
    for sigma1 in SIGMA1_LIST:
        for sigma2 in SIGMA2_LIST:
            # Initialize the Bayesian network with current prior parameters
            model = BayesianNet(INPUT_DIM, hidden_dim=800, output_dim=10,
                                pi=pi, sigma1=sigma1, sigma2=sigma2).to(device)
            optimizer = optim.Adam(model.parameters(), lr=LR_BBB)
            best_test_acc = 0.0

            # Epoch progress bar
            epoch_desc = f"BBB π={pi:.2f} σ₁={sigma1:.1e} σ₂={sigma2:.1e}"
            for epoch in trange(NUM_EPOCHS_BBB, desc=epoch_desc, leave=True):
                model.train()
                running_loss = 0.0
                total = 0
                correct = 0
                num_batches = len(train_loader)

                # Training loop (no tqdm on batches)
                for data, target in train_loader:
                    data, target = data.to(device), target.to(device)
                    optimizer.zero_grad()
                    output = model(data)                              # Bayesian forward pass
                    nll = criterion(output, target)                   # Negative log-likelihood
                    kl_loss = model.kl / N_TRAIN                      # Normalized KL divergence
                    loss = nll + kl_loss
                    loss.backward()
                    optimizer.step()

                    running_loss += loss.item()
                    preds = output.argmax(dim=1)
                    correct += (preds == target).sum().item()
                    total += target.size(0)

                train_acc = correct / total * 100

                # Evaluation on test set (using deterministic weights: μ)
                model.eval()
                correct_test = 0
                total_test = 0
                with torch.no_grad():
                    for data, target in test_loader:
                        data, target = data.to(device), target.to(device)
                        x = F.relu(model.bl1(data, sample=False))
                        x = F.relu(model.bl2(x, sample=False))
                        logits = model.bl3(x, sample=False)
                        preds = logits.argmax(dim=1)
                        correct_test += (preds == target).sum().item()
                        total_test += target.size(0)
                test_acc = correct_test / total_test * 100
                if test_acc > best_test_acc:
                    best_test_acc = test_acc

                # Print metrics at the end of the epoch
                print(
                    f"  Epoch {epoch+1:02d}/{NUM_EPOCHS_BBB} | "
                    f"Train Loss={running_loss/num_batches:.4f} | "
                    f"Train Acc={train_acc:.2f}% | "
                    f"Test Acc={test_acc:.2f}%"
                )

            print(f"✅ BBB π={pi:.2f}, σ₁={sigma1:.1e}, σ₂={sigma2:.1e} → "
                  f"Best Test Acc = {best_test_acc:.2f}%")
            results_bbb.append((pi, sigma1, sigma2, best_test_acc))

BBB π=0.75 σ₁=1.4e-01 σ₂=2.5e-03:   0%|          | 1/600 [00:05<55:59,  5.61s/it]

  Epoca 01/600 | Train Loss=16.2261 | Train Acc=10.13% | Test Acc=35.54%


BBB π=0.75 σ₁=1.4e-01 σ₂=2.5e-03:   0%|          | 2/600 [00:10<54:03,  5.42s/it]

  Epoca 02/600 | Train Loss=15.9518 | Train Acc=13.33% | Test Acc=28.44%


BBB π=0.75 σ₁=1.4e-01 σ₂=2.5e-03:   0%|          | 3/600 [00:16<53:49,  5.41s/it]

...

  Epoca 598/600 | Train Loss=5.1118 | Train Acc=95.95% | Test Acc=97.57%


BBB π=0.75 σ₁=1.4e-01 σ₂=2.5e-03: 100%|█████████▉| 599/600 [44:54<00:04,  4.22s/it]

  Epoca 599/600 | Train Loss=5.1150 | Train Acc=95.88% | Test Acc=97.50%


BBB π=0.75 σ₁=1.4e-01 σ₂=2.5e-03: 100%|██████████| 600/600 [44:58<00:00,  4.50s/it]

  Epoca 600/600 | Train Loss=5.1196 | Train Acc=95.86% | Test Acc=97.46%
✅ BBB π=0.75, σ₁=1.4e-01, σ₂=2.5e-03 → Best Test Acc = 97.75%


### 📈 Final Evaluation Summary

This section prints the final test results for all models:

#### ✅ Bayes by Backprop
- Displays best test accuracy for each combination of prior parameters:
  - π (mixture weight)
  - σ₁ and σ₂ (standard deviations of the Gaussian mixture prior)

#### ✅ TensorFlow Baseline Models
- Shows test accuracy for:
  - **Standard SGD** (no regularization)
  - **SGD + Dropout** (Dropout probability = 0.5)

The results allow direct comparison between Bayesian and non-Bayesian training strategies on the MNIST dataset.

In [ ]:
# Print results for Bayes by Backprop
print("\n📊 Bayes by Backprop Results (π, σ₁, σ₂, Best Test Accuracy):")
for pi, s1, s2, acc in results_bbb:
    print(f"  π={pi:.2f}, σ₁={s1:.4e}, σ₂={s2:.4e}  →  Best Test Acc = {acc:.2f}%")

# Convert TensorFlow baseline accuracies from [0, 1] to percentages
acc_baseline_sgd = acc_sgd * 100
acc_baseline_do  = acc_do  * 100

# Print results for baseline TensorFlow models
print("\n📊 Baseline TF Results:")
print(f"  SGD without regularization       → Best Test Acc = {acc_baseline_sgd:.2f}%")
print(f"  SGD + Dropout (p=0.5)             → Best Test Acc = {acc_baseline_do:.2f}%")


📊 Risultati Bayes by Backprop (π, σ₁, σ₂, Best Test Accuracy):
  π=0.75, σ₁=1.3534e-01, σ₂=2.4788e-03  →  Best Test Acc = 97.75%

📊 Risultati Baseline TF:
  SGD senza regularizzazione       → Best Test Acc = 97.20%
  SGD + Dropout (p=0.5)             → Best Test Acc = 97.39%
